In [ ]:
from google.colab import drive
import os

drive.mount('/content/GoogleDrive', force_remount=True)
path = '/content/GoogleDrive/My Drive/TULKH'
os.chdir(path)
!ls

Mounted at /content/GoogleDrive
data.txt


# Ant Cology Optimization

In [ ]:
import random
from collections import defaultdict, namedtuple

f = open('data.txt', 'r')

# Define a namedtuple for classes
ClassInfo = namedtuple('ClassInfo', ['periods', 'teacher', 'students'])
# Define the maximum number of iterations and other ACO parameters
MAX_ITERATIONS = 100
NUM_ANTS = 10
EVAPORATION_RATE = 0.5
PHEROMONE_IMPORTANCE = 1.0
HEURISTIC_IMPORTANCE = 2.0

def read_input():
    # Read N and M
    N, M = map(int, f.readline().split())

    # Read class info
    classes = []
    for _ in range(N):
        periods, teacher, students = map(int, f.readline().split())
        classes.append(ClassInfo(periods, teacher, students))

    # Read room capacities
    room_capacities = list(map(int, f.readline().split()))

    return N, M, classes, room_capacities

def initialize_pheromones(N, M, periods):
    # Initialize pheromone levels to 1.0 for all possible (class, period, room) combinations
    pheromones = defaultdict(lambda: 1.0)
    return pheromones

def heuristic_value(class_info, room_capacity):
    # Heuristic value can be higher if the room fits the class size well
    if class_info.students <= room_capacity:
        return 1.0
    return 0.0

def select_next_move(pheromones, heuristic, possible_moves):
    # Calculate probabilities based on pheromones and heuristics
    probabilities = []
    for move in possible_moves:
        pheromone = pheromones[move]
        heuristic_val = heuristic[move]
        probabilities.append((pheromone ** PHEROMONE_IMPORTANCE) * (heuristic_val ** HEURISTIC_IMPORTANCE))

    total = sum(probabilities)
    probabilities = [p / total for p in probabilities]

    # Roulette wheel selection
    r = random.uniform(0, 1)
    cumulative_probability = 0.0
    for i, prob in enumerate(probabilities):
        cumulative_probability += prob
        if cumulative_probability >= r:
            return possible_moves[i]

    return possible_moves[-1]

def construct_solution(pheromones, heuristic, N, M, classes, room_capacities):
    solution = []
    remaining_classes = list(range(N))
    constraints = {
        'teacher': defaultdict(set),
        'room': defaultdict(set)
    }

    while remaining_classes:
        class_idx = remaining_classes.pop(0)
        class_info = classes[class_idx]

        possible_moves = []
        for period in range(60):
            for room in range(M):
                if class_info.students <= room_capacities[room] and period not in constraints['teacher'][class_info.teacher]:
                    possible_moves.append((class_idx, period, room))

        if not possible_moves:
            continue

        move = select_next_move(pheromones, heuristic, possible_moves)
        solution.append(move)

        class_idx, period, room = move
        for p in range(period, period + classes[class_idx].periods):
            constraints['teacher'][classes[class_idx].teacher].add(p)
            constraints['room'][room].add(p)

    return solution

def update_pheromones(pheromones, solutions):
    # Evaporate pheromones
    for key in pheromones.keys():
        pheromones[key] *= (1 - EVAPORATION_RATE)

    # Increase pheromones for the best solutions
    for solution in solutions:
        quality = len(solution)
        for move in solution:
            pheromones[move] += quality

def evaluate_solution(solution):
    return len(solution)

def aco_schedule_classes(N, M, classes, room_capacities):
    pheromones = initialize_pheromones(N, M, 60)
    heuristic = defaultdict(lambda: 1.0)  # For simplicity, set heuristic to 1.0 initially

    best_solution = None
    best_quality = -1

    for iteration in range(MAX_ITERATIONS):
        solutions = []

        for _ in range(NUM_ANTS):
            solution = construct_solution(pheromones, heuristic, N, M, classes, room_capacities)
            solutions.append(solution)

            quality = evaluate_solution(solution)
            if quality > best_quality:
                best_quality = quality
                best_solution = solution

        update_pheromones(pheromones, solutions)

    return best_solution

def main():
    N, M, classes, room_capacities = read_input()
    best_schedule = aco_schedule_classes(N, M, classes, room_capacities)

    print(len(best_schedule))
    for class_idx, period, room in best_schedule:
        print(class_idx + 1, period + 1, room + 1)

if __name__ == "__main__":
    main()


9
1 17 1
2 36 2
3 13 2
4 56 1
5 10 1
6 49 1
8 19 1
9 22 1
10 34 2


# Genetic Algorithm

In [ ]:
import random

# Constants
NUM_DAYS = 5
SLOTS_PER_DAY = 12
TOTAL_SLOTS = NUM_DAYS * SLOTS_PER_DAY

def read_data_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

        # Đọc N và M
        N, M = map(int, lines[0].split())

        # Đọc thông tin các lớp
        classes = []
        for i in range(1, N + 1):
            t, g, s = map(int, lines[i].split())
            classes.append((t, g, s))

        # Đọc sức chứa các phòng
        rooms = list(map(int, lines[N + 1].split()))

    return N, M, classes, rooms

def initialize_population(pop_size, classes, rooms):
    population = []
    for _ in range(pop_size):
        individual = []
        for class_id, (t, g, s) in enumerate(classes):
            valid_rooms = [r_id + 1 for r_id, capacity in enumerate(rooms) if s <= capacity]
            if valid_rooms:
                time_slot = random.randint(1, TOTAL_SLOTS)
                room_id = random.choice(valid_rooms)
                individual.append((class_id, time_slot, room_id))
            else:
                individual.append((class_id, -1, -1))  # Đánh dấu không thể xếp
        population.append(individual)
    return population

def fitness(individual, classes, rooms):
    score = 0
    teacher_schedule = {}
    room_occupancy = {}

    for class_id, time_slot, room_id in individual:
        if time_slot == -1 and room_id == -1:
            continue  # Bỏ qua các lớp không thể xếp

        t, g, s = classes[class_id]
        if s <= rooms[room_id - 1]:
            if g not in teacher_schedule:
                teacher_schedule[g] = set()
            if room_id not in room_occupancy:
                room_occupancy[room_id] = set()

            if time_slot not in teacher_schedule[g] and time_slot not in room_occupancy[room_id]:
                teacher_schedule[g].add(time_slot)
                room_occupancy[room_id].add(time_slot)
                score += 10  # Tăng điểm cho xếp hợp lệ
            else:
                score -= 5  # Phạt khi có xung đột
        else:
            score -= 5  # Phạt khi vượt quá sức chứa phòng

    # Phạt thêm cho các lớp không được xếp
    unassigned_classes = sum(1 for class_id, time_slot, room_id in individual if time_slot == -1 and room_id == -1)
    score -= unassigned_classes * 10

    return score

def tournament_selection(population, fitnesses, k=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitnesses)), k)
        winner = max(tournament, key=lambda x: x[1])
        selected.append(winner[0])
    return selected

def uniform_crossover(parent1, parent2):
    child1, child2 = [], []
    for gene1, gene2 in zip(parent1, parent2):
        if random.random() < 0.5:
            child1.append(gene1)
            child2.append(gene2)
        else:
            child1.append(gene2)
            child2.append(gene1)
    return child1, child2

def mutate(individual, classes, rooms, mutation_rate):
    for idx, (class_id, time_slot, room_id) in enumerate(individual):
        if random.random() < mutation_rate:
            t, g, s = classes[class_id]
            valid_rooms = [r_id + 1 for r_id, capacity in enumerate(rooms) if s <= capacity]
            if valid_rooms:
                time_slot = random.randint(1, TOTAL_SLOTS)
                room_id = random.choice(valid_rooms)
                individual[idx] = (class_id, time_slot, room_id)
            else:
                individual[idx] = (class_id, -1, -1)  # Đánh dấu không thể xếp
    return individual

def genetic_algorithm(classes, rooms, pop_size=100, generations=1000, initial_mutation_rate=0.01, tournament_size=3):
    population = initialize_population(pop_size, classes, rooms)
    for generation in range(generations):
        fitnesses = [fitness(ind, classes, rooms) for ind in population]
        if generation % 100 == 0:
            print(f"Generation {generation}, Best Fitness: {max(fitnesses)}")

        selected_population = tournament_selection(population, fitnesses, tournament_size)
        next_population = []
        for i in range(0, len(selected_population), 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i + 1] if i + 1 < len(selected_population) else selected_population[0]
            child1, child2 = uniform_crossover(parent1, parent2)
            next_population.extend([child1, child2])

        mutation_rate = initial_mutation_rate / (1 + generation / generations)  # Tỷ lệ đột biến thích ứng
        next_population = [mutate(ind, classes, rooms, mutation_rate) for ind in next_population]

        population = next_population

    best_individual = max(population, key=lambda ind: fitness(ind, classes, rooms))
    return best_individual

# Đọc dữ liệu từ file
filename = 'data.txt'  # Cập nhật tên file đầu vào của bạn
N, M, classes, rooms = read_data_from_file(filename)

# Chạy thuật toán di truyền
best_schedule = genetic_algorithm(classes, rooms)

# Kết quả
print("Lịch trình tốt nhất tìm thấy:")
for class_id, time_slot, room_id in best_schedule:
    if time_slot != -1 and room_id != -1:
        print(f"{class_id + 1} {time_slot} {room_id}")

Generation 0, Best Fitness: 80
Generation 100, Best Fitness: 80
Generation 200, Best Fitness: 80
Generation 300, Best Fitness: 80
Generation 400, Best Fitness: 80
Generation 500, Best Fitness: 80
Generation 600, Best Fitness: 80
Generation 700, Best Fitness: 80
Generation 800, Best Fitness: 80
Generation 900, Best Fitness: 80
Lịch trình tốt nhất tìm thấy:
1 3 2
2 2 2
3 23 1
4 22 1
5 18 2
6 6 2
8 51 1
9 4 2
10 40 1
